Results for leg queries and ranking features as well as prf for determining whether top ranked document contains legislation that is relevant to automatically apply filtering methods.

In [ ]:
from plotlib.loaders import *
from plotlib.plotters import *
from phdconf import config 
from phdconf.config import *

import pandas as pd
import copy
import subprocess

In [ ]:
queries = load_queries(AUS_LEG_TOPIC_PATH)
broad, specific = load_query_types(queries)

paths = ['leg-casename.run', 'leg-citation.run', 'leg-count.run', 'leg-date.run', 'leg-query.run', 'leg-sec-count.run', 'leg-specific-sec-count.run', 'leg-interp.run', 'leg-comb.run', 'auspdfs-unigram_dir_mu_1050.00.run', 'filtered-phrasestop-unigram_dir_mu_1050.00.run']
paths = ['all-leg-queries-'+x for x in paths]
names = ['casenames', 'citations', 'counts', 'date', 'query', 'section', 'specific_section', 'interp', 'comb', 'pdf', 'text']
displays = ['Casename', 'Citation', 'Legislation', 'Date', 'Query', 'Section', 'Specific Sec', 'Interp', 'Combined', 'PDF', 'Cleaned']

In [ ]:
# def create_folds(queries, k: int):
#     l = len(queries)
#     q_set = set(queries)
#     inds = [x for x in range(l)]
#     np.random.shuffle(inds)
#     queries = np.array(queries)
#     queries = queries[inds]
    
#     avg = l / float(k)
#     parts = []
#     last = 0.0
    
#     test = []

#     while last < l:
#         test.append(list(queries[int(last):int(last + avg)]))
#         last += avg
     
#     for i in range(len(parts)):
#         train.append()
    
#     train = []
#     for i in range(len(test)):
#         train.append(list(q_set.difference(test[i])))
    
#     return train, test
    
# train, test = create_folds(list(queries.keys()), 5)

In [ ]:
# # write folds to file 
# with open('ausleg-folds.txt', 'w') as f:
#     for tr, te in zip(train, test):
#         f.write('{0} {1}\n'.format(tr, te))

In [ ]:
# dfs = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, os.path.join(config.BASE_DIR, 'dirichlet_prior'), paths)
dfs = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, os.path.join(BASE_DIR, 'legislative'), paths, per_query=True)

In [ ]:
def calc_sig(dfs, comps, names, metrics):
    d = pd.DataFrame(columns=dfs[0].columns)
    all_ind = set()
    for df in dfs:
        all_ind.update(df.index)
        
    for i in range(len(dfs)): 
        ind = all_ind - set(dfs[i].index)
        ncol = len(dfs[i].columns)
        for x in ind: 
            dfs[i].loc[x] = [0.0] * ncol
        dfs[i].sort_index(inplace=True)
    
    symbols = ['*', '\dagger', '\cdot', '\textdollar', '\textyen', '\textcent', '\texteuro', '\text', '\cdot', '\circ', '\centerdot', '\diamond']
    comp_res = []
    for c in comps: 
        res = {}
        for m in metrics:
            res[m] = dfs[c][m]
        comp_res.append((c, res))

    for i, df in enumerate(dfs):
        av = df.mean()
        for j, res in enumerate(comp_res):
            if i <= res[0]: 
                continue
            for m in metrics:
#                 print(res[m])
#                 print(df[m])
                p = stats.ttest_rel(res[1][m], df[m]).pvalue
                if p < 0.05: 
                    sym = symbols[j]
                    if p < 0.01:
                        sym = sym*2
                    if isinstance(av[m], str):
                        av[m] = av[m][:-2] + sym + "}$"
                    else:
                        av[m] = "{0:.4f}$^{{".format(av[m]) + sym + "}$"
                elif isinstance(av[m], float):
                    av[m] = "{0:.4f}$^{{}}$".format(av[m])

        d.loc[names[i]] = av
    print(d.round(4).filter(metrics, axis='columns').rename(metrics, axis='columns').to_latex(escape=False))
    return d.filter(metrics, axis='columns').rename(metrics, axis='columns')
                    

metrics = {}
metrics['set_P'] = 'P'
metrics['set_recall'] = 'R'
metrics['set_F_.5'] = 'F_{0.5}'
all_leg_df = calc_sig(dfs, [4, 9, 10], displays, metrics)  

In [ ]:
stats.ttest_rel(dfs[9]['recip_rank'], dfs[10]['recip_rank'])

In [ ]:
sub_df = all_leg_df.reindex(['Query', 'PDF', 'Cleaned'])
sub_df = sub_df.rename(index={'PDF': '\texttt{PDF}', 'Cleaned': '\texttt{cleaned}', 'Query': '\texttt{bool}'})
# write_table('tables/ausl-leg-bool', bold_max(sub_df).to_latex(escape=False))

In [ ]:
metrics = copy.copy(config.METRIC_NAMES)
del metrics['unjudged@20']
calc_sig(dfs[:-2], [4], displays[:-2], metrics)  

In [ ]:
paths = ['leg-casename.run', 'leg-citation.run', 'leg-count.run', 'leg-date.run', 'leg-query.run', 'leg-sec-count.run', 'leg-specific-sec-count.run', 'leg-interp.run', 'leg-comb.run', 'auspdfs-unigram_dir_mu_1050.00.run', 'filtered-phrasestop-unigram_dir_mu_1050.00.run']
paths = ['all-leg-queries-'+x for x in paths]
names = ['casenames', 'citations', 'counts', 'date', 'query', 'section', 'specific_section', 'interp', 'comb', 'pdf', 'text']
    
# names[4]
inter = Interpolater(os.path.join(BASE_DIR, 'legislative/', paths[4]), normalize=True)

dfs = []
for d in paths[:4]+paths[5:-4]: 
    print(d)
    interped_dfs = []
    for _lambda in np.arange(0, 1.0, 0.01):
        inter.interpolate(os.path.join(BASE_DIR, 'legislative/', d), _lambda, 'tmp.run')
        interped_dfs.append(load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', ['tmp.run'], per_query=True)[0])
    dfs.append(interped_dfs)

In [ ]:
tt_folds = read_folds(AUS_FOLDS)

In [ ]:
base_query = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, '', [os.path.join(BASE_DIR, 'legislative', paths[4])], per_query=True)[0]

In [ ]:
metrics = copy.copy(config.METRIC_NAMES)

ntlm_df = pd.DataFrame(columns=metrics)

for ab, runs in zip(names[2:4]+names[5:-4], dfs[2:]):
    print(ab)
    ntlm_cross = cross_validation(runs, tt_folds, metrics, base_query)
    ntlm_df.loc[ab] = ntlm_cross[0]

In [ ]:
ntlm_df.loc['$R$'] = base_query.mean().round(4)
ntlm_df = ntlm_df.reindex(['$R$', 'counts', 'section', 'specific_section', 'date'])
print(ntlm_df.rename(columns=metrics).drop(['Unjudged@20', 'R@100'],axis='columns').to_latex(escape=False))

In [ ]:
ausl_leg_df = ntlm_df.reindex(['$R$', 'counts', 'section'])
ausl_leg_df = ausl_leg_df.rename(index={'counts': 'leg', 'section': 'sec'})
# write_table('tables/ausl-leg-ranking', bold_max(ausl_leg_df).rename(columns=metrics).drop(['Unjudged@20', 'R@100'],axis='columns').to_latex(escape=False))

In [ ]:
ausl_date_df = ntlm_df.reindex(['$R$', 'date'])
# write_table('tables/ausl-date-ranking', bold_max(ausl_date_df).rename(columns=metrics).drop(['Unjudged@20', 'R@100'],axis='columns').to_latex(escape=False))

In [ ]:
def get_missed(run_name: str, path: str = os.path.join(BASE_DIR, 'legislative')):    
    retrieved = {}
    with open(f'{path}/{run_name}.run') as f:
        for line in f: 
            parts = line.strip().split()
            if parts[0] not in retrieved:
                retrieved[parts[0]] = set()
            retrieved[parts[0]].add(parts[2])
            
    relevant = {}
    with open(config.AUS_QREL_PATH) as f:
        for line in f:
            parts = line.strip().split()
            if parts[0] not in retrieved:
                continue
            if parts[0] not in relevant:
                relevant[parts[0]] = set()
            if parts[3] == "1":
                relevant[parts[0]].add(parts[2])
                
                
    for topic in relevant.keys():
        items = copy.copy(relevant[topic])
        for doc in items:
            if doc in retrieved[topic]: 
                relevant[topic].remove(doc)
    
    return(relevant)
    
missed = get_missed('all-leg-queries-leg-query')


# not cite - 33
# 2 - 2006QCA155, 2003FCA50
# 17 - 2014FCA765, 
# 35 - 2003FCAFC109, 2014QSC281
# 47 - 2005FCA494, 2008FCAFC170
# 55 - 2009FCA1502, 2013FCA1038, 2011FCA1159, 2013QCA104, 2015FCA741
# 60 - 2015QSC149, 2014QCA183
# 61 - 2013QCA161, 2016FCA243, 2002QCA225, 2012QCA100, 2009QSC417, 2016QCA215, 2010QSC122, 2008FCA0865, 2014FCA585
# 66 - 2006QSC349
# 77 - 2005QCA277, 2010FCA769, 2005FCA918, 2010FCA285, 2018FCA33
# 78 - 2010FCA285, 2005FCA918, 2018FCA33
# 86 - 2011QDC213

# reason for this is might be appeal or costs decision

# cite prev leg - 4
# 36 - 2001QSC211
# 38 - 2004FCA5
# 92 - 2011QSC307, 2009QCA75

# cite other sec from same act or another act - 15
# 17 - 2011FCA1063
# 55 - 2019QSC144
# 60 - 2001QSC444, 2007QCA185, 2007QSC92, 2008QSC4, 2011QSC88, 2012QDC73, 2006QSC274, 2004QSC272, 2009QSC192, 
# 78 - 2018QCA79, 2007QSC262
# 96 - 2018FCA657, 2018FCA1405

In [ ]:
# missed_standard = get_missed('all-leg-queries-leg-linked-rep')

In [ ]:
# for k, v in missed.items():
#     print(k, v)
dfs = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, os.path.join(BASE_DIR, 'legislative/'), paths, per_query=True)

In [ ]:
topics = ['legislative-queries', 'case-topics']
expansion_paths = ['leg-exp.run', 'leg-filtered-exp.run', 'leg-linked-all.run', 'leg-linked-rep.run', 'leg-linked-same.run']
expansion_paths = ['all-leg-queries-'+x for x in expansion_paths]

results = load_dfs(config.AUS_QREL_PATH, config.AUS_REL_LEVEL, os.path.join(BASE_DIR, 'legislative/'), expansion_paths, per_query=True)

In [ ]:
metrics['set_P'] = 'P'
metrics['set_recall'] = 'R'
metrics['set_F_.5'] = 'F_{0.5}'
leg_filter_df = calc_sig(dfs[-3:]+results, [0, 1, 2], displays[-3:]+['leg-exp', 'leg-filtered-exp', 'leg-linked-all', 'leg-linked-rep', 'leg-linked-same'], metrics)    

In [ ]:
leg_filter_df = leg_filter_df.rename(index={'PDF': '\texttt{PDF}', 'Cleaned': '\texttt{cleaned}', 'Combined': '\texttt{bool}', 
                                        'leg-linked-all': 'all-link', 'leg-linked-rep': 'rep-link', 'leg-linked-same': 'same-link'})
leg_filter_df = leg_filter_df.applymap(lambda x: round(x, 4) 
    if isinstance(x, (int, float)) else x)
# write_table('tables/ausl-leg-exp', bold_max(leg_filter_df.round(4)).rename(columns=metrics).to_latex(escape=False))

In [ ]:
ndf = leg_filter_df.reindex(['\texttt{bool}', '\texttt{cleaned}', '\texttt{PDF}', 'leg-exp'])
ndf = ndf.applymap(lambda x: round(x, 4) if isinstance(x, (int, float)) else x)
# write_table('tables/ausl-leg-term-exp', bold_max(ndf).rename(columns=metrics).to_latex(escape=False))

In [ ]:
metrics = copy.copy(config.METRIC_NAMES)
del metrics ['unjudged@20']
calc_sig(dfs+results, [4, 9, 10], displays+['leg-exp', 'leg-filtered-exp', 'leg-linked-all', 'leg-linked-rep', 'leg-linked-same'], metrics)    

In [ ]:
results[-1][(results[-1]-dfs[-3])['set_recall'] < 0]

In [ ]:
dfs[-3].loc[38]

In [ ]:
miss_b = get_missed('all-leg-queries-leg-linked-all') 
miss_a = get_missed('all-leg-queries-leg-exp') 
# first is missing from baseline 
# for k, v in miss_a.items():
#     if len(v) > 0 or len(miss_b[k]) > 0:
#         print(k, v, '|', miss_b[k])
# print('-'*20)
# miss_b = get_missed('legislative-queries-leg-linked-same') 
# miss_a = get_missed('legislative-queries-leg-exp') 
# for k, v in miss_a.items():
#     if len(v) > 0 or len(miss_b[k]) > 0:
#         print(k, v, '|', miss_b[k])
# print('-'*20)
# miss_b = get_missed('legislative-queries-leg-linked-rep') 
# miss_a = get_missed('legislative-queries-leg-exp') 
# for k, v in miss_a.items():
#     if len(v) > 0 or len(miss_b[k]) > 0:
#         print(k, v, '|', miss_b[k])
# print('-'*20)

In [ ]:
# get_missed('all-leg-queries-leg-exp') 

In [ ]:
def get_num_results(path: str):
    res = {}
    with open(path) as f:
        for line in f:
            topic = line.split()[0]
            v = res.get(topic, 0)
            res[topic] = v + 1
                
    return res

In [ ]:
# get_num_results(os.path.join(BASE_DIR, 'legislative/', 'case-topics-leg-exp.run'))

In [ ]:
# get_num_results(os.path.join(BASE_DIR, 'legislative/', 'legislative-queries-flattened-unigram_dir_mu_2400.00.run'))

# clear files

In [ ]:
def load_prf(path: str): 
    with open(path) as f:
        data = json.load(f)
        return data
        
prf_data = load_prf('../features/leg-prf.json')

In [ ]:
secs = []
legs = []
for i in range(0, 4):
    secs.append([])
    legs.append([])

for section in prf_data['sections']:
    q_leg = set()
    q_sec = []
    
    if section in prf_data['Found']:
        q_leg = set(prf_data['Found'][section][0])
        if prf_data['Found'][section][1] is not None:
            q_sec = set(prf_data['Found'][section][1])
    
    for i in range(0, 4):
        if prf_data['legislation'][section][i] is None:
            continue
            
        for l in prf_data['legislation'][section][i].items():
            if l[0] == 'act' or l[0] == 'acts':
                continue
            legs[i].append((section, l[0], l[1], l[0] in q_leg))
            
        for s in prf_data['sections'][section][i].items():
            secs[i].append((section, s[0], s[1], s[0] in q_sec))

#         legs = sorted(prf_data['legislation'][section][i].items(), key=lambda x: x[1], reverse=True)
#         secs = sorted(prf_data['sections'][section][i].items(), key=lambda x: x[1], reverse=True)
                    
    
#     break

In [ ]:
print(prf_data['Found'].keys())
print(set(prf_data['sections'].keys()).difference(set(prf_data['Found'].keys())))
print(len(prf_data['sections'].keys()))
print(prf_data['legislation'].keys())

In [ ]:
sec_df = pd.DataFrame(data=secs[0], columns=['query', 'item', 'count', 'in_query'])
f, ax = plt.subplots(1)
sec_scatter = sns.scatterplot(x='count', y='query',
              hue='in_query',
              data=sec_df)

f.set_size_inches(16, 4)
ax.set_xlabel('Frequency', fontsize=15)
ax.axes.yaxis.set_visible(False)
ax.axes.tick_params(labelsize=12)
ax.axes.yaxis.set_visible(False)
ax.get_legend().remove()
# f.savefig('figures/aus-sec-prf.pdf')

In [ ]:
leg_df = pd.DataFrame(data=legs[0], columns=['query', 'item', 'count', 'in_query'])
f, ax = plt.subplots(1)
leg_scatter = sns.scatterplot(x='count', y='query',
              hue='in_query',
              data=leg_df, ax=ax)

f.set_size_inches(16, 4)
# ax.axes.xaxis.set_visible(False)
ax.set_xlabel('Frequency', fontsize=15)
ax.axes.yaxis.set_visible(False)
ax.axes.tick_params(labelsize=12)
ax.get_legend().remove()
# f.savefig('figures/aus-leg-prf.pdf')